![](https://raw.githubusercontent.com/batistagroup/ChemSpaceAL/packaging/media/logo.png)

This Colab notebook allows you to easily implement [ChemSpaceAL v2.0.0](https://github.com/batistagroup/ChemSpaceAL/). For more details, check out the associated [preprint on arXiv](https://arxiv.org/abs/2309.05853). Please feel free to start any discussion or raise any issues on our [GitHub](https://github.com/batistagroup/ChemSpaceAL/).

![](https://raw.githubusercontent.com/batistagroup/ChemSpaceAL/packaging/media/toc_figure.jpg)

In [5]:
%%capture
#@title Set Up Notebook
#@markdown Press the *Play* button to install ChemSpaceAL and its dependencies

!rm -r ChemSpaceAL
!git clone --single-branch --branch packaging https://github.com/batistagroup/ChemSpaceAL
!git clone https://github.com/Liuhong99/Sophia.git
!pip install ChemSpaceAL/.

import ChemSpaceAL
from ChemSpaceAL import InitializeWorkspace
from ChemSpaceAL import Configuration
from ChemSpaceAL import Dataset
from ChemSpaceAL import Model
from ChemSpaceAL import Training
from ChemSpaceAL import Generation
from ChemSpaceAL import Sampling
from ChemSpaceAL import ALConstruction
base_path = None

import os

# Initialize the Workspace

As implemented in the paper, one iteration of ChemSpaceAL takes ~22 hours, of which 20 are spent on docking. To ensure no data is lost due to accidental termination of Colab sessions, we strongly recommend connecting Google Drive and creating a dedicated folder for storing results of runs.

If you do not execute the cell below, all files will be saved within local sessions and will be lost if the session terminates!

Please note that this package has been optimized for running multiple AL iterations. Practically, this means that the code may seem daunting to you at first, but once you get familiar with it, you can analyze the results of scoring from some AL iteration and launch the docking for the next iteration within 30 min! Feel free [to reach out to us](https://github.com/batistagroup/ChemSpaceAL/) if you need help with getting started!

In [6]:
#@title Specify (base) path for storing results
# @markdown make sure your path ends with a "/"
base_path = "/content/drive/MyDrive/ChemSpaceAL-runs/" # @param {type:"string"}

from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


## Create subfolders for storing results
By default, the following folder structure will be created

```
ChemSpaceAL-runs
├── 1_Pretraining
│   ├── dataset_folder: datasets
│   ├── desc_folder: datasets_descriptors
│   └── weight_folder: model_weights
├── 2_Generation
├── 3_Sampling
│   ├── desc_folder: generations_descriptors
│   ├── pca_folder: pca_weights
│   ├── kmeans_folder: kmeans_objects
│   └── clustering_folder: clusterings
├── 4_Scoring
│   ├── target_folder: binding_targets
│   ├── candidate_folder: sampled_mols
│   ├── pose_folder: binding_poses
│   └── score_folder: scored_dataframes
└── 5_ActiveLearning
    ├── train_folder: training_sets
    ├── desc_folder: trainingset_descriptors
    └── weight_folder: model_weights
```

This structure is specified by `InitializeWorkspace.FOLDER_STRUCTURE`. The values to keys `*_folder` are the folder names. You're more than welcome them to rename if you want and pass a new folder_structure dict to `InitializeWorkspace.create_folders` as an optional parameter `folder_structure=`.

In [7]:
#@title create subfolders
#@markdown By default, the following folder structure will be created
if base_path is None:
  base_path = os.getcwd() + "/runs/"
InitializeWorkspace.create_folders(base_path=base_path)


    will create folders at base_path='/content/drive/MyDrive/ChemSpaceAL-runs/'


In [30]:
#@title Download (if you want) dataset/weights
#@markdown note these files will be placed into appropriate folders created above
downloadDataset = False # @param {type:"boolean"}
downloadModelWeights = True # @param {type:"boolean"}
downloadPCAweights = True # @param {type:"boolean"}
script = '''#!/bin/bash
'''
remote_source = "https://files.ischemist.com/ChemSpaceAL/publication_runs/"
if downloadDataset:
  f1 = "1_Pretraining/datasets/combined_train.csv.gz"
  f2 = "1_Pretraining/datasets/combined_valid.csv.gz"
  script += f"curl -o {base_path}{f1} {remote_source}{f1}\n"
  script += f"curl -o {base_path}{f2} {remote_source}{f2}\n"
if downloadModelWeights:
  f1 = "1_Pretraining/datasets_descriptors/combined_train.yaml"
  f2 = "1_Pretraining/model_weights/model7_al0_ch1.pt"
  script += f"curl -o {base_path}{f1} {remote_source}{f1}\n"
  script += f"curl -o {base_path}{f2} {remote_source}{f2}\n"
if downloadPCAweights:
  f1 = "3_Sampling/pca_weights/scaler_pca_combined_n120.pkl"
  script += f"curl -o {base_path}{f1} {remote_source}{f1}\n"
with open("fetch.bash", "w") as f:
  f.write(script)
!bash fetch.bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1357  100  1357    0     0   6238      0 --:--:-- --:--:-- --:--:--  6253
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 24.9M  100 24.9M    0     0  45.5M      0 --:--:-- --:--:-- --:--:-- 45.5M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  208k  100  208k    0     0  1056k      0 --:--:-- --:--:-- --:--:-- 1057k


# Active Learning Iteration

First, we have to initialize a global config. A brief explanation of some of the parameters.

`cycle_prefix`, `al_iteration`, and `cycle_suffix` are used to compose filenames for all results and intermediary output. For example, with default parameters, all filenames will start as `model0_al0_ch1`. We recommend changing `cycle_prefix` for different pretrained generative models, changing `cycle_suffix` for different settings of AL, and you **must** not forget to increment `al_iteration` as you progress through AL cycles.

`training_fname` and `validation_fname`

## Setting parameters

In [8]:
config = Configuration.Config(
    base_path=base_path,
    cycle_prefix="model0",
    cycle_suffix="ch1",
    al_iteration=1, # use 0 for pretraining
    training_fname="combined_train.csv.gz",
    validation_fname="combined_valid.csv.gz",
    slice_data=1,
    verbose=True, # will print every important decision that's going to be made
)
# The following fills two lists (or you could provide them manually as optional parameters)
# `previously_scored_mols` - a list of strings of paths to previously scored molecules
# `previous_al_train_sets` - a list of strings of paths to AL training sets from previous iterations
# these lists are needed to
# - assess how many of generated molecules are repeated from AL training sets
# - make sure already scored molecules are not sampled again for docking
# notably, if you do not mess with the default naming system, these lists will be filled for you automatically
# once you change al_iteration to a non-zero value!
config.set_previous_arrays()

--- The following previously scored molecules were set:
     4_Scoring/scored_dataframes/model0_al0_ch1.csv
--- The following previously constructed Active Learning sets were set:
     5_ActiveLearning/training_sets/model0_al0_ch1.csv


## Pretraining

In [34]:
# mode can be set to "Pretraining" or "Active Learning".
# In "Pretraining", an output is a list of two dataset objects corrresponding to (train, valid) partitions
# In "Active Learning", an output is a single dataset object corresponding to an AL training set
datasets = Dataset.load_data(config=config, mode="Pretraining")

In [13]:
# You can also overwrite `learning_rate`, `lr_warmup` (a boolean of whether to do lr warmup),
# For a full list of available parameters run help(config.set_training_parameters)
config.set_training_parameters(mode="Pretraining", epochs=1)

--- The following training parameters were set:
    number of epochs: 1
    learning rate: 0.0003
    learning warmup enabled? True
    model weights will be saved to:               1_Pretraining/model_weights/model0_al1_ch1.pt                                   
    dataset descriptors will be loaded from:      1_Pretraining/datasets_descriptors/combined_train.yaml                          
  . note: wandb_project_name and wandb_runname were not provided, you can ignore this message if you don't plan to log runs to wandb


In [36]:
# model objects and trainer objects are returned in case you want to do something with them
model, trainer = Training.train_GPT(
    config=config,
    training_dataset=datasets[0],
    validation_dataset=datasets[1]
)

/usr/local/lib/python3.10/dist-packages/torch/cuda/amp/grad_scaler.py:125: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
epoch 1 iter 0: train loss 3.00481. lr 3.000000e-05: 100%|██████████| 1/1 [00:03<00:00,  3.38s/it]


## Generation

In [14]:
config.set_generation_parameters(
    target_criterion="force_number_filtered", # or you could choose `force_number_unique` or `force_number_completions`
    force_filters="ADMET+FGs", # could choose `ADMET` for no restriction on functional groups or simply remove this parameter
    target_number=1,
)

--- The following generation parameters were set:
    target number: 1 unique canonical smiles that pass filters
    batch size: 64 & temperature: 1.0
    the following filters will be applied: ADMET+FGs
    model weights will be loaded from:            1_Pretraining/model_weights/model0_al1_ch1.pt                                   
    dataset descriptors will be loaded from:      1_Pretraining/datasets_descriptors/combined_train.yaml                          
    generated completions will be saved to:       2_Generation/model0_al1_ch1_completions.csv                                     
    unique canonic smiles will be saved to:       2_Generation/model0_al1_ch1_unique_smiles.csv                                   
    generation metrics will be saved to:          2_Generation/model0_al1_ch1_metrics.txt                                         
    filtered molecules will be saved to:          2_Generation/model0_al1_ch1_filtered_smiles.csv                                 
    The fo

In [ ]:
Generation.generate_smiles(config) # this runs generation of SMILES
Generation.characterize_generated_molecules(config) # this runs an analysis of # unique, valid, and novel molecules

## Sampling

In [15]:
config.set_sampling_parameters(
    n_clusters=10,
    samples_per_cluster=2,
    pca_fname="scaler_pca_combined_n120.pkl",
)

--- The following sampling parameters were set:
    number of clusters: 10
    samples per cluster: 2
    descriptors mode: mix
    descriptors will be saved to:                      3_Sampling/generations_descriptors/model0_al1_ch1.pkl                           
    PCA will be loaded from:                           3_Sampling/pca_weights/scaler_pca_combined_n120.pkl                             
    KMeans Objects will be saved to:                   3_Sampling/kmeans_objects/model0_al1_ch1_k10.pkl                                
    cluster to molecules mapping will be saved to:     3_Sampling/clusterings/model0_al1_ch1_cluster_to_mols.pkl                       
    sampled molecules will be saved to:                4_Scoring/sampled_mols/model0_al1_ch1_sampled20.csv                             


In [ ]:
Sampling.calculate_descriptors(config)
mols = Sampling.project_into_pca_space(config)
Sampling.cluster_and_sample(mols=mols, config=config, n_iter=1)

## Scoring

In [9]:
%%capture
#@title Install Docking Software (DiffDock)
#@markdown diffdock is pretty heavy and has a lot of dependencies, so we only install it when we need it (and we don't during pretraining, for example)

import torch

print(torch.__version__)
!pip uninstall torch-scatter torch-sparse torch-geometric torch-cluster  --y
!pip install torch-scatter -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-cluster -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install git+https://github.com/pyg-team/pytorch_geometric.git  --quiet

try:
    import biopandas
except:
    !pip install pyg==0.7.1 --quiet
    !pip install pyyaml==6.0 --quiet
    !pip install scipy==1.7.3 --quiet
    !pip install networkx==2.6.3 --quiet
    !pip install biopython==1.79 --quiet
    !pip install rdkit-pypi==2022.03.5 --quiet
    !pip install e3nn==0.5.0 --quiet
    !pip install spyrmsd==0.5.2 --quiet
    !pip install pandas==1.5.3 --quiet
    !pip install biopandas==0.4.1 --quiet

if not os.path.exists("/content/DiffDock"):
    os.chdir('/content')
    !git clone https://github.com/gcorso/DiffDock.git
    os.chdir('/content/DiffDock')
    !git checkout a6c5275
    os.chdir('/content')

if not os.path.exists("/content/DiffDock/esm"):
    os.chdir('/content/DiffDock')
    !git clone https://github.com/facebookresearch/esm
    os.chdir('/content/DiffDock/esm')
    !git checkout ca8a710
    !sudo pip install -e .
    os.chdir('/content/DiffDock')
    os.chdir('/content')

from rdkit import Chem
import shutil
import os
import pandas as pd
from tqdm import tqdm

In [2]:
from ChemSpaceAL.Docking import get_top_poses

2.1.0+cu121


In [16]:
config.set_scoring_parameters(
    protein_path="HNH_processed.pdb",
)

--- The following scoring parameters were set:
    Reminder that docking poses will be written to 4_Scoring/binding_poses/                                                        
    protein will be loaded from                   4_Scoring/binding_targets/HNH_processed.pdb                                     
    poses will be saved to                        4_Scoring/binding_poses/model0_al1_ch1/                                         
    and scored molecules will be saved to         4_Scoring/scored_dataframes/model0_al1_ch1.csv                                  
    The following prolif interaction weights will be used:
    |    Hydrophobic: 2.5, HBDonor: 3.5, HBAcceptor: 3.5, Anionic: 7.5, Cationic: 7.5,
    |    CationPi: 2.5, PiCation: 2.5, VdWContact: 1.0, XBAcceptor: 3.0,
    |    XBDonor: 3.0, FaceToFace: 3.0, EdgeToFace: 1.0, MetalDonor: 3.0,
    |    MetalAcceptor: 3.0


### Docking

In [13]:
get_top_poses(
    ligands_csv=config.cycle_temp_params["path_to_sampled"],
    protein_pdb_path=config.cycle_temp_params["path_to_protein"],
    save_pose_path=config.cycle_temp_params["path_to_poses"]
)

100%|██████████| 20/20 [00:00<00:00, 1515.06it/s]


### Counting attractive interaction scores

In [18]:
from ChemSpaceAL import Scoring
ligand_scores = Scoring.score_ligands(config)

100%|██████████| 21/21 [00:07<00:00,  2.94it/s]


In [ ]:
Scoring.parse_and_prepare_diffdock_data(
    ligand_scores=ligand_scores,
    config=config
)

## Active Learning

In [17]:
config.set_active_learning_parameters(
    selection_mode="threshold", probability_mode="linear", threshold=11, training_size=10
)

--- The following AL training set construction parameters were set:
    the training set will be constructed to have 10 molecules
    of which 5 will be selected from the top scoring molecules defined by the following parameters:
        molecules with score above 11 will be selected
    the remaining 5 molecules will be selected from high-scoring clusters according to the following parameters:
        the following probability mode will be used: linear
    the training set will be saved to             5_ActiveLearning/training_sets/model0_al1_ch1.csv                               


In [ ]:
ALConstruction.construct_al_training_set(config=config, do_sampling=True)

In [10]:
al_ds = Dataset.load_data(config=config, mode="Active Learning")

Will load AL training set from 5_ActiveLearning/training_sets/model0_al0_ch1.csv


In [18]:
config.set_training_parameters(mode="Active Learning", epochs=1)

--- The following training parameters were set:
    number of epochs: 1
    learning rate: 3e-05
    learning warmup enabled? False
    model weights will be loaded from:            1_Pretraining/model_weights/model0_al0_ch1.pt                                   
    model weights will be saved to:               5_ActiveLearning/model_weights/model0_al1_ch1.pt                                
    dataset descriptors will be loaded from:      1_Pretraining/datasets_descriptors/combined_train.yaml                          
  . note: wandb_project_name and wandb_runname were not provided, you can ignore this message if you don't plan to log runs to wandb


In [ ]:
model, trainer = Training.train_GPT(
    config=config,
    training_dataset=al_ds,
)